# Machine Learning and its application in Muon Tomography

Machine Learning inspired techniques can be applied to Muon Tomography to resolve many of associated problems. The two problems of interest to us are *low resolution tomograms* and *data inefficiency*. In this notebook, I am discussing the use of ML as applied to MT to solve problem of data inefficiency. 

## Data Inefficiency Problem

![pic](mt.jpeg)


We define "good muon events" as events with 4 by 4 coincidence hits. Such events have defined spatial points in both top and bottom trays. For example, for the hit on the right the information
$$ A (x_1,y_1) $$
$$ B (x_2,y_2) $$

are known. Due to limited capabilities of our instruments, such events are rare (less than $40\%$ withour best set up) with many events following the trend of the hit on the left. 

$$ C (x_3,y_3) $$
$$ D (x_4,?) $$

We know that $y_4 \in (y_{min}, y_{max})$ and is discernable by $y_{res}$ which is the limited hardware spatial resolution of the telescope. Now, the question is what this $y_4$ may be? 

Once we know how to address this question, we can reduce our data redundancy drastically and improve our overall resolution and efficiency of our schema.

## Track reconstruction using non-perfect events

Each event is assigned a probability that describes the likelihood of using information from that event to reconstruct the muon track. Such a scheme means that events with 4 by 4 coincidence hits have a probability of 1 and events with no hits have probability 0. Events with 3 by 4 hits are thus also assigned a probability.

The way we assign this probability requires the use of some neural networks - *RNN's and LSTM's*. The idea is as follows:

1. Contextualize the entire dataset into a RNN framework
2. Calculate key statistics of overall dataset and individual events
3. Locate events with missing information (i.e. not perfect 4 by 4 coincidence hits)
4. Assign probabilities to candidates for the missing information using wholistic statistics, TDC data, and the angular distribution of muons.
4. Use RNN's and LSTM's to predict the missing datum in 3 by 4 events using the probabilistic approach.


## Probability distribution of the muon hit of a single spatial dimension

Using the coordinate with full information *(C in our case)* and the other spatial coordinate ($x_4$), we assign probabilites to each $y_i \in (y_{min}, y_{max})$. Formally, it can be represented using Bayes' theorem.

$$ P(y_4 = y_i | x_3,y_3,x_4)  $$

Now, the question is how do we create such a probability distribution? What controlled/measured factors can help us make such decisions?

### No information known

When we know previous/extra information about the muon hit, one can imagine the distribution is uniform on the space of the spatial dimension.

$$ P(y_4 = y_i | x_3,y_3,x_4) \sim \mathcal{U} (y_{min}, y_{max})  $$

### One spatial coordinate and one single spatial dimension known

The distribution for such a case is much more complicated. The following are some ways of generating the distribution.

#### Exact Aggregate Data Approach

In this approach, we simply assign the probabilities based on the most likely value from the set of "good" events that share the exact coordinates with the event under consideration. The probability simply becomes a fraction of the two aggregate type of events.

- $P(y_4 = y_i) = \frac{\text{# of events with} y_i,x_3,y_3,x_4}{\text{# of events with} x_3,y_3,x_4} $
    
#### Similar Aggregate Data Approach
    
We do a similar thing like the **Exact Aggregate Data Approach** with the added consideration of neighbors to influence the statistic.
    
#### Shift of TDC Approach  

Since, we have demonstrated that TDC values are correlated with the transverse distance of the muon hit along the scintilator bar. It is possible to extrapolate 2D information from such values. In this approach, we do just that to determine the missing spatial dimension.
- $P(y_4 = y_i) = $

#### Most Common Angle Approach

In this approach, we select the missing spatial dimension such that the angle resembles the most common angle. 

- $P(y_4 = y_i) = $ 


The best method should be some combination of all such approaches. This is where ML comes in. The ML scheme would work to solve for the coffecients that of such a weighted sum to **maximize** resolution of tomogram and **minimize** data inefficiency.